In [1]:
#importando o arquivo, base de dados
import os
os.chdir('C:\\Users\\Usuario\\Desktop\\dna')
data1= open('splice.data.txt', 'r')

#passando o arquivo para um list, com as classes convertidas para int, converte as series para int
f= data1.readlines()
for i in range(len(f)):
    f[i]= f[i].split(',')
for i in range(len(f)):
    del(f[i][1])
for i in range(len(f)):
    f[i][1]=f[i][1].split()[0]
c=[]
for i in range(len(f)):
    b=[]
    if f[i][0]=='EI':
        b.append(0)
    elif f[i][0]=='IE':
        b.append(1)
    elif f[i][0]=='N':
        b.append(2)
    a=[]
    for j in range(len(f[i][1])):
        if f[i][1][j]=='A':
            a.append(0)
        elif f[i][1][j]=='C':
            a.append(1)
        elif f[i][1][j]=='G':
            a.append(2)
        elif f[i][1][j]=='T':
            a.append(3)
        else:
            a.append(8)
    b.append(a)
    c.append(b)
cdata=[]
ctarg=[]

for i in range(len(c)):
    cdata.append(c[i][1])
    ctarg.append((c[i][0]))
    


In [2]:
#importando ferramentas
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn import tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier


from sklearn.feature_selection import SelectFromModel

#ensemble
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import classification_report,confusion_matrix

In [3]:
#contando quantos integrantes de cada classe na base de dados
j1=0
j2=0
j3=0
for i in range(len(ctarg)):
    if(ctarg[i]==0):
        j1=j1+1
    elif(ctarg[i]==1):
        j2=j2+1
    elif(ctarg[i]==2):
        j3=j3+1
print(j1,j2,j3)

767 768 1655


In [25]:
#Voting Classifier with weight, pesos definidos pela media da porcentagem de acerto de cada classe em uma 10-fold cross-validation
#primeiro separa a base em 90% treino,  10% para testes, de maneira que se repita 10 vezes, tendo testes para toda a base
class0 = cdata[0:767]
class1 = cdata[767:1535]
class2 = cdata[1535:3190]
class0T = ctarg[0:767]
class1T = ctarg[767:1535]
class2T = ctarg[1535:3190]

clf1 = SVC(gamma='auto')
clf2 = KNeighborsClassifier(n_neighbors=2)
clf3 = tree.DecisionTreeClassifier()
clf4 = GaussianNB()
VotingWithWeightsPredictions=[]
predictions0= []
predictions1= []
predictions2= []
i=0
while i<10:
    #separacao da base
    dataTeste= class0[int(i*len(class0)/10):int((i+1)*len(class0)/10)]+class1[int(i*len(class1)/10):int((i+1)*len(class1)/10)]+class2[int(i*len(class2)/10):int((i+1)*len(class2)/10)]
    c0= class0.copy()
    c1= class1.copy()
    c2= class2.copy()
    del(c0[int(i*len(class0)/10):int((i+1)*len(class0)/10)])
    del(c1[int(i*len(class1)/10):int((i+1)*len(class1)/10)])
    del(c2[int(i*len(class2)/10):int((i+1)*len(class2)/10)])
    data= c0 + c1 + c2
    targTeste= class0T[int(i*len(class0T)/10):int((i+1)*len(class0T)/10)]+class1T[int(i*len(class1T)/10):int((i+1)*len(class1T)/10)]+class2T[int(i*len(class2T)/10):int((i+1)*len(class2T)/10)]
    c0T= class0T.copy()
    c1T= class1T.copy()
    c2T= class2T.copy()
    del(c0T[int(i*len(class0T)/10):int((i+1)*len(class0T)/10)])
    del(c1T[int(i*len(class1T)/10):int((i+1)*len(class1T)/10)])
    del(c2T[int(i*len(class2T)/10):int((i+1)*len(class2T)/10)])
    targ= c0T + c1T + c2T
    
    #calculando os pesos, 10-fold cross-validation no conjunto de treino apenas
    j=0
    M1W= []      #list para armezar os pesos do modelo 1
    M2W= []
    M3W= []
    M4W= []
    while j<10:
        #Separacao do conjunto de treino em 10-fold 
        dataPTeste =c0[int(j*len(c0)/10):int((j+1)*len(c0)/10)]+c1[int(j*len(c1)/10):int((j+1)*len(c1)/10)]+c2[int(j*len(c2)/10):int((j+1)*len(c2)/10)]
        c0P= c0.copy()
        c1P= c1.copy()
        c2P= c2.copy()
        del(c0P[int(j*len(c0)/10):int((j+1)*len(c0)/10)])
        del(c1P[int(j*len(c1)/10):int((j+1)*len(c1)/10)])
        del(c2P[int(j*len(c2)/10):int((j+1)*len(c2)/10)])
        dataP = c0P + c1P + c2P
        targPTeste =c0T[int(j*len(c0T)/10):int((j+1)*len(c0T)/10)]+c1T[int(j*len(c1T)/10):int((j+1)*len(c1T)/10)]+c2T[int(j*len(c2T)/10):int((j+1)*len(c2T)/10)]
        c0TP= c0T.copy()
        c1TP= c1T.copy()
        c2TP= c2T.copy()
        del(c0TP[int(j*len(c0T)/10):int((j+1)*len(c0T)/10)])
        del(c1TP[int(j*len(c1T)/10):int((j+1)*len(c1T)/10)])
        del(c2TP[int(j*len(c2T)/10):int((j+1)*len(c2T)/10)])
        targP = c0TP + c1TP + c2TP
        
        #construcao dos modelos, predicao do teste
        clf1.fit(dataP,targP)
        clf2.fit(dataP,targP)
        clf3.fit(dataP,targP)
        clf4.fit(dataP,targP)
        
        T1=clf1.predict(dataPTeste)
        T2=clf2.predict(dataPTeste)
        T3=clf3.predict(dataPTeste)
        T4=clf4.predict(dataPTeste)
        
        #confusion matrix e calculo dos pesos por porcentagem de acerto dos palpites
        cm1 = confusion_matrix(targPTeste, T1)
        cm2 = confusion_matrix(targPTeste, T2)
        cm3 = confusion_matrix(targPTeste, T3)
        cm4 = confusion_matrix(targPTeste, T4)
        
        T1weights= [cm1[0][0]/(cm1[0][0] + cm1[1][0] + cm1[2][0]), cm1[1][1]/(cm1[0][1] + cm1[1][1] + cm1[2][1]), cm1[2][2]/(cm1[0][2] + cm1[1][2] + cm1[2][2])]
        T2weights= [cm2[0][0]/(cm2[0][0] + cm2[1][0] + cm2[2][0]), cm2[1][1]/(cm2[0][1] + cm2[1][1] + cm2[2][1]), cm2[2][2]/(cm2[0][2] + cm2[1][2] + cm2[2][2])]
        T3weights= [cm3[0][0]/(cm3[0][0] + cm3[1][0] + cm3[2][0]), cm3[1][1]/(cm3[0][1] + cm3[1][1] + cm3[2][1]), cm3[2][2]/(cm3[0][2] + cm3[1][2] + cm3[2][2])]
        T4weights= [cm4[0][0]/(cm4[0][0] + cm4[1][0] + cm4[2][0]), cm4[1][1]/(cm4[0][1] + cm4[1][1] + cm4[2][1]), cm4[2][2]/(cm4[0][2] + cm4[1][2] + cm4[2][2])]
        
        M1W.append(T1weights)
        M2W.append(T2weights)
        M3W.append(T3weights)
        M4W.append(T4weights)
        
        j= j+1
    #media dos pesos de cada modelo
    M1Wm =[0,0,0]
    M2Wm =[0,0,0]
    M3Wm =[0,0,0]
    M4Wm =[0,0,0]
    for v in range(len(M1W)):
        M1Wm[0] += M1W[v][0]
        M1Wm[1] += M1W[v][1]
        M1Wm[2] += M1W[v][2]
        M2Wm[0] += M2W[v][0]
        M2Wm[1] += M2W[v][1]
        M2Wm[2] += M2W[v][2]
        M3Wm[0] += M3W[v][0]
        M3Wm[1] += M3W[v][1]
        M3Wm[2] += M3W[v][2]
        M4Wm[0] += M4W[v][0]
        M4Wm[1] += M4W[v][1]
        M4Wm[2] += M4W[v][2]
    M1Wm= [M1Wm[0]/10, M1Wm[1]/10, M1Wm[2]/10]
    M2Wm= [M2Wm[0]/10, M2Wm[1]/10, M2Wm[2]/10]
    M3Wm= [M3Wm[0]/10, M3Wm[1]/10, M3Wm[2]/10]
    M4Wm= [M4Wm[0]/10, M4Wm[1]/10, M4Wm[2]/10]
    
    #fit model e votacao
    clf1.fit(data,targ)
    clf2.fit(data,targ)
    clf3.fit(data,targ)
    clf4.fit(data,targ)

    T1=clf1.predict(dataTeste)
    T2=clf2.predict(dataTeste)
    T3=clf3.predict(dataTeste)
    T4=clf4.predict(dataTeste)
    
    
    
    for k in range(len(targTeste)):
        predictClass= [0,0,0]
        predictClass[T1[k]]= predictClass[T1[k]] + M1Wm[T1[k]]
        predictClass[T2[k]]= predictClass[T2[k]] + M2Wm[T2[k]]
        predictClass[T3[k]]= predictClass[T3[k]] + M3Wm[T3[k]]
        predictClass[T4[k]]= predictClass[T4[k]] + M4Wm[T4[k]]
        if(targTeste[k]==0):
            predictions0.append(np.argmax(predictClass))
        if(targTeste[k]==1):
            predictions1.append(np.argmax(predictClass))
        if(targTeste[k]==2):
            predictions2.append(np.argmax(predictClass))
    
    i= i+1
VotingWithWeightsPredictions= predictions0 + predictions1 + predictions2
    
#Verificando a precisao
c0=0
p0=0
c1=0
p1=0
c2=0
p2=0
for j in range(len(ctarg)):
        if(ctarg[j]==0):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p0=p0+1
            c0= c0 +1
        if(ctarg[j]==1):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p1=p1+1
            c1= c1 +1
        if(ctarg[j]==2):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p2=p2+1
            c2= c2 +1

print("Accuracy: %0.2f  , ei: %0.2f , ie: %0.2f, n: %0.2f" % ((p0/c0+ p1/c1 + p2/c2)/3, p0/c0, p1/c1, p2/c2))
conf_mat = confusion_matrix(ctarg, VotingWithWeightsPredictions)
print('Confusion:\n', conf_mat, '\n')

Accuracy: 0.92  , ei: 0.91 , ie: 0.92, n: 0.94
Confusion:
 [[ 701   37   29]
 [  21  704   43]
 [  39   61 1555]] 



In [23]:
M1Wm =[0,0,0]
for v in range(len(M1W)):
    M1Wm[0] += M1W[v][0]
    M1Wm[1] += M1W[v][1]
    M1Wm[2] += M1W[v][2]
M1Wm

[8.212144444929827, 8.024811039599502, 8.785426796724852]

In [16]:
#Voting Classifier with weight, pesos definidos pela porcentagem de acerto de cada classe
#primeiro separa a base em 80% treino, 10% para obter os pesos, 10% para testes, de maneira que se repita 10 vezes, tendo testes para toda a base
j1=0
j2=0
j3=0
for i in range(len(ctarg)):
    if(ctarg[i]==0):
        j1=j1+1
    elif(ctarg[i]==1):
        j2=j2+1
    elif(ctarg[i]==2):
        j3=j3+1

clf1 = SGDClassifier(loss="hinge", penalty="l2", max_iter=5, tol= 1e-3)
clf2 = KNeighborsClassifier(n_neighbors=2)
clf3 = tree.DecisionTreeClassifier()
clf4 = GaussianNB()
clf5 = SVC(gamma='auto')
VotingWithWeightsPredictions=[]
predictions0= []
predictions1= []
predictions2= []
i=0
ii=0
while i < 9:
    data= cdata[0:int(i*(j1/10))]+cdata[int((2+i)*(j1/10)):j1]+cdata[j1:j1+int(i*(j2/10))]+cdata[j1+int((2+i)*(j2/10)):j1+j2]+cdata[j1+j2:j1+j2+int(i*(j3/10))]+cdata[j1+j2+int((2+i)*(j3/10)):j1+j2+j3]
    dataPesos= cdata[int(i*(j1/10)):int((1+i)*(j1/10))]+cdata[j1+int(i*(j2/10)):j1+int((1+i)*(j2/10))]+cdata[j1+j2+int(i*(j3/10)): j1+j2+int((1+i)*(j3/10))]
    dataTeste= cdata[int((1+i)*(j1/10)):int((2+i)*(j1/10))]+cdata[j1+int((1+i)*(j2/10)):j1+int((2+i)*(j2/10))]+cdata[j1+j2+int((1+i)*(j3/10)): j1+j2+int((2+i)*(j3/10))]
    targ= ctarg[0:int(i*(j1/10))]+ctarg[int((2+i)*(j1/10)):j1]+ctarg[j1:j1+int(i*(j2/10))]+ctarg[j1+int((2+i)*(j2/10)):j1+j2]+ctarg[j1+j2:j1+j2+int(i*(j3/10))]+ctarg[j1+j2+int((2+i)*(j3/10)):j1+j2+j3] 
    targPesos= ctarg[int(i*(j1/10)):int((1+i)*(j1/10))]+ctarg[j1+int(i*(j2/10)):j1+int((1+i)*(j2/10))]+ctarg[j1+j2+int(i*(j3/10)): j1+j2+int((1+i)*(j3/10))]
    targTeste= ctarg[int((1+i)*(j1/10)):int((2+i)*(j1/10))]+ctarg[j1+int((1+i)*(j2/10)):j1+int((2+i)*(j2/10))]+ctarg[j1+j2+int((1+i)*(j3/10)): j1+j2+int((2+i)*(j3/10))]
    if(ii==0):
        dpcopy= dataPesos.copy()
        dataPesos= dataTeste.copy()
        dataTeste= dpcopy.copy()
        tpcopy= targPesos.copy()
        targPesos= targTeste.copy()
        targTeste= tpcopy.copy()
    clf1.fit(data,targ)
    clf2.fit(data,targ)
    clf3.fit(data,targ)
    clf4.fit(data,targ)
    clf5.fit(data,targ)
    P1= clf1.predict(dataPesos)
    P2=clf2.predict(dataPesos)
    P3=clf3.predict(dataPesos)
    P4=clf4.predict(dataPesos)
    P5=clf5.predict(dataPesos)
    T1=clf1.predict(dataTeste)
    T2=clf2.predict(dataTeste)
    T3=clf3.predict(dataTeste)
    T4=clf4.predict(dataTeste)
    T5=clf5.predict(dataTeste)
    class0= 0
    P1class0=0
    P2class0=0
    P3class0=0
    P4class0=0
    P5class0=0
    class1=0
    P1class1=0
    P2class1=0
    P3class1=0
    P4class1=0
    P5class1=0
    class2=0
    P1class2=0
    P2class2=0
    P3class2=0
    P4class2=0
    P5class2=0
    for j in range(len(targPesos)):
        if(targPesos[j]==0):
            if(targPesos[j]==P1[j]):
                P1class0= P1class0 +1
            if(targPesos[j]==P2[j]):
                P2class0= P2class0 +1
            if(targPesos[j]==P3[j]):
                P3class0= P3class0 +1
            if(targPesos[j]==P4[j]):
                P4class0= P4class0 +1
            if(targPesos[j]==P5[j]):
                P5class0= P5class0 +1
            class0= class0 +1
        if(targPesos[j]==1):
            if(targPesos[j]==P1[j]):
                P1class1= P1class1 +1
            if(targPesos[j]==P2[j]):
                P2class1= P2class1 +1
            if(targPesos[j]==P3[j]):
                P3class1= P3class1 +1
            if(targPesos[j]==P4[j]):
                P4class1= P4class1 +1
            if(targPesos[j]==P5[j]):
                P5class1= P5class1 +1
            class1= class1 +1
        if(targPesos[j]==2):
            if(targPesos[j]==P1[j]):
                P1class2= P1class2 +1
            if(targPesos[j]==P2[j]):
                P2class2= P2class2 +1
            if(targPesos[j]==P3[j]):
                P3class2= P3class2 +1
            if(targPesos[j]==P4[j]):
                P4class2= P4class2 +1
            if(targPesos[j]==P5[j]):
                P5class2= P5class2 +1
            class2= class2 +1
    T1class0weight = P1class0/class0
    T1class1weight = P1class1/class1
    T1class2weight = P1class2/class2
    T1weights= [T1class0weight, T1class1weight, T1class2weight]
    T2class0weight = P2class0/class0
    T2class1weight = P2class1/class1
    T2class2weight = P2class2/class2
    T2weights= [T2class0weight, T2class1weight, T2class2weight]
    T3class0weight = P3class0/class0
    T3class1weight = P3class1/class1
    T3class2weight = P3class2/class2
    T3weights= [T3class0weight, T3class1weight, T3class2weight]
    T4class0weight = P4class0/class0
    T4class1weight = P4class1/class1
    T4class2weight = P4class2/class2
    T4weights= [T4class0weight, T4class1weight, T4class2weight]
    T5class0weight = P5class0/class0
    T5class1weight = P5class1/class1
    T5class2weight = P5class2/class2
    T5weights= [T5class0weight, T5class1weight, T5class2weight]
    
    for k in range(len(targTeste)):
        predictClass= [0,0,0]
        predictClass[T1[k]]= predictClass[T1[k]] + T1weights[T1[k]]
        predictClass[T2[k]]= predictClass[T2[k]] + T2weights[T2[k]]
        predictClass[T3[k]]= predictClass[T3[k]] + T3weights[T3[k]]
        predictClass[T4[k]]= predictClass[T4[k]] + T4weights[T4[k]]
        predictClass[T5[k]]= predictClass[T5[k]] + T5weights[T5[k]]
        if(targTeste[k]==0):
            predictions0.append(np.argmax(predictClass))
        if(targTeste[k]==1):
            predictions1.append(np.argmax(predictClass))
        if(targTeste[k]==2):
            predictions2.append(np.argmax(predictClass))
    if(ii==0):
        ii= ii+1
        i= i-1
    i= i+1
VotingWithWeightsPredictions= predictions0 + predictions1 + predictions2
    

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

In [17]:
#Verificando a precisao
c0=0
p0=0
c1=0
p1=0
c2=0
p2=0
for j in range(len(ctarg)):
        if(ctarg[j]==0):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p0=p0+1
            c0= c0 +1
        if(ctarg[j]==1):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p1=p1+1
            c1= c1 +1
        if(ctarg[j]==2):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p2=p2+1
            c2= c2 +1

print("Accuracy: %0.2f  , ei: %0.2f , ie: %0.2f, n: %0.2f" % ((p0/c0+ p1/c1 + p2/c2)/3, p0/c0, p1/c1, p2/c2))
conf_mat = confusion_matrix(ctarg, VotingWithWeightsPredictions)
print('Confusion:\n', conf_mat, '\n')

Accuracy: 0.90  , ei: 0.90 , ie: 0.91, n: 0.90
Confusion:
 [[ 690   49   28]
 [  45  698   25]
 [  75   88 1492]] 



In [35]:
print("Accuracy: %0.2f  , ei: %0.2f , ie: %0.2f, n: %0.2f" % ((p0/c0+ p1/c1 + p2/c2)/3, p0/c0, p1/c1, p2/c2))

Accuracy: 0.95  , ei: 0.96 , ie: 0.95, n: 0.95


In [18]:
#Voting Classifier with weight, pesos definidos pela porcentagem de acerto de cada classe em relacao ao numero de vezes que o palpite foi nesta classe
#primeiro separa a base em 80% treino, 10% para obter os pesos, 10% para testes, de maneira que se repita 10 vezes, tendo testes para toda a base

j1=0
j2=0
j3=0
for i in range(len(ctarg)):
    if(ctarg[i]==0):
        j1=j1+1
    elif(ctarg[i]==1):
        j2=j2+1
    elif(ctarg[i]==2):
        j3=j3+1

clf1 = SGDClassifier(loss="hinge", penalty="l2", max_iter=5, tol= 1e-3)
clf2 = KNeighborsClassifier(n_neighbors=2)
clf3 = tree.DecisionTreeClassifier()
clf4 = GaussianNB()
clf5 = SVC(gamma='auto')
VotingWithWeightsPredictions=[]
predictions0= []
predictions1= []
predictions2= []
i=0
ii=0
while i < 9:
    data= cdata[0:int(i*(j1/10))]+cdata[int((2+i)*(j1/10)):j1]+cdata[j1:j1+int(i*(j2/10))]+cdata[j1+int((2+i)*(j2/10)):j1+j2]+cdata[j1+j2:j1+j2+int(i*(j3/10))]+cdata[j1+j2+int((2+i)*(j3/10)):j1+j2+j3]
    dataPesos= cdata[int(i*(j1/10)):int((1+i)*(j1/10))]+cdata[j1+int(i*(j2/10)):j1+int((1+i)*(j2/10))]+cdata[j1+j2+int(i*(j3/10)): j1+j2+int((1+i)*(j3/10))]
    dataTeste= cdata[int((1+i)*(j1/10)):int((2+i)*(j1/10))]+cdata[j1+int((1+i)*(j2/10)):j1+int((2+i)*(j2/10))]+cdata[j1+j2+int((1+i)*(j3/10)): j1+j2+int((2+i)*(j3/10))]
    targ= ctarg[0:int(i*(j1/10))]+ctarg[int((2+i)*(j1/10)):j1]+ctarg[j1:j1+int(i*(j2/10))]+ctarg[j1+int((2+i)*(j2/10)):j1+j2]+ctarg[j1+j2:j1+j2+int(i*(j3/10))]+ctarg[j1+j2+int((2+i)*(j3/10)):j1+j2+j3] 
    targPesos= ctarg[int(i*(j1/10)):int((1+i)*(j1/10))]+ctarg[j1+int(i*(j2/10)):j1+int((1+i)*(j2/10))]+ctarg[j1+j2+int(i*(j3/10)): j1+j2+int((1+i)*(j3/10))]
    targTeste= ctarg[int((1+i)*(j1/10)):int((2+i)*(j1/10))]+ctarg[j1+int((1+i)*(j2/10)):j1+int((2+i)*(j2/10))]+ctarg[j1+j2+int((1+i)*(j3/10)): j1+j2+int((2+i)*(j3/10))]
    if(ii==0):
        dpcopy= dataPesos.copy()
        dataPesos= dataTeste.copy()
        dataTeste= dpcopy.copy()
        tpcopy= targPesos.copy()
        targPesos= targTeste.copy()
        targTeste= tpcopy.copy()
    clf1.fit(data,targ)
    clf2.fit(data,targ)
    clf3.fit(data,targ)
    clf4.fit(data,targ)
    clf5.fit(data,targ)
    P1= clf1.predict(dataPesos)
    P2=clf2.predict(dataPesos)
    P3=clf3.predict(dataPesos)
    P4=clf4.predict(dataPesos)
    P5=clf5.predict(dataPesos)
    T1=clf1.predict(dataTeste)
    T2=clf2.predict(dataTeste)
    T3=clf3.predict(dataTeste)
    T4=clf4.predict(dataTeste)
    T5=clf5.predict(dataTeste)
    
    cm1 = confusion_matrix(targPesos, P1)
    cm2 = confusion_matrix(targPesos, P2)
    cm3 = confusion_matrix(targPesos, P3)
    cm4 = confusion_matrix(targPesos, P4)
    cm5 = confusion_matrix(targPesos, P5)
    
    
    T1weights= [cm1[0][0]/(cm1[0][0] + cm1[1][0] + cm1[2][0]), cm1[1][1]/(cm1[0][1] + cm1[1][1] + cm1[2][1]), cm1[2][2]/(cm1[0][2] + cm1[1][2] + cm1[2][2])]
    T2weights= [cm2[0][0]/(cm2[0][0] + cm2[1][0] + cm2[2][0]), cm2[1][1]/(cm2[0][1] + cm2[1][1] + cm2[2][1]), cm2[2][2]/(cm2[0][2] + cm2[1][2] + cm2[2][2])]
    T3weights= [cm3[0][0]/(cm3[0][0] + cm3[1][0] + cm3[2][0]), cm3[1][1]/(cm3[0][1] + cm3[1][1] + cm3[2][1]), cm3[2][2]/(cm3[0][2] + cm3[1][2] + cm3[2][2])]
    T4weights= [cm4[0][0]/(cm4[0][0] + cm4[1][0] + cm4[2][0]), cm4[1][1]/(cm4[0][1] + cm4[1][1] + cm4[2][1]), cm4[2][2]/(cm4[0][2] + cm4[1][2] + cm4[2][2])]
    T5weights= [cm5[0][0]/(cm5[0][0] + cm5[1][0] + cm5[2][0]), cm5[1][1]/(cm5[0][1] + cm5[1][1] + cm5[2][1]), cm5[2][2]/(cm5[0][2] + cm5[1][2] + cm5[2][2])]
    
    
    for k in range(len(targTeste)):
        predictClass= [0,0,0]
        predictClass[T1[k]]= predictClass[T1[k]] + T1weights[T1[k]]
        predictClass[T2[k]]= predictClass[T2[k]] + T2weights[T2[k]]
        predictClass[T3[k]]= predictClass[T3[k]] + T3weights[T3[k]]
        predictClass[T4[k]]= predictClass[T4[k]] + T4weights[T4[k]]
        predictClass[T5[k]]= predictClass[T5[k]] + T5weights[T5[k]]
        if(targTeste[k]==0):
            predictions0.append(np.argmax(predictClass))
        if(targTeste[k]==1):
            predictions1.append(np.argmax(predictClass))
        if(targTeste[k]==2):
            predictions2.append(np.argmax(predictClass))
    if(ii==0):
        ii= ii+1
        i= i-1
    i= i+1
VotingWithWeightsPredictions= predictions0 + predictions1 + predictions2
    

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

In [19]:
#Verificando a precisao
c0=0
p0=0
c1=0
p1=0
c2=0
p2=0
for j in range(len(ctarg)):
        if(ctarg[j]==0):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p0=p0+1
            c0= c0 +1
        if(ctarg[j]==1):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p1=p1+1
            c1= c1 +1
        if(ctarg[j]==2):
            if(ctarg[j]==VotingWithWeightsPredictions[j]):
                p2=p2+1
            c2= c2 +1
print("Accuracy: %0.2f  , ei: %0.2f , ie: %0.2f, n: %0.2f" % ((p0/c0+ p1/c1 + p2/c2)/3, p0/c0, p1/c1, p2/c2))
conf_mat = confusion_matrix(ctarg, VotingWithWeightsPredictions)
print('Confusion:\n', conf_mat, '\n')

Accuracy: 0.90  , ei: 0.90 , ie: 0.90, n: 0.91
Confusion:
 [[ 688   37   42]
 [  36  694   38]
 [  56   94 1505]] 



In [15]:
print('Voting Classifier')
clf1 = SGDClassifier(loss="hinge", penalty="l2", max_iter=5, tol= 1e-3)
clf2 = KNeighborsClassifier(n_neighbors=2)
clf3 = tree.DecisionTreeClassifier()
clf4 = GaussianNB()
clf5 = SVC(gamma='auto')
clf = VotingClassifier(estimators=[('gbc', clf1), ('ada', clf2), ('DTc', clf3), ('gNB', clf4), ('svc', clf5)], voting='hard')
scores = cross_val_score(clf, cdata, ctarg, cv=10)
y_pred = cross_val_predict(clf, cdata, ctarg, cv=10)
conf_mat = confusion_matrix(ctarg, y_pred)
print('\nScores:',scores)
print("Accuracy: %0.2f (+/- %0.2f) , ei: %0.2f , ie: %0.2f, n: %0.2f" % (scores.mean(), scores.std() * 2, conf_mat[0][0]/sum(conf_mat[0]), conf_mat[1][1]/sum(conf_mat[1]),conf_mat[2][2]/sum(conf_mat[2])))
print('Confusion:\n', conf_mat, '\n')

Voting Classifier


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-


Scores: [0.890625   0.89375    0.89375    0.884375   0.921875   0.86206897
 0.90595611 0.8836478  0.89274448 0.87697161]
Accuracy: 0.89 (+/- 0.03) , ei: 0.93 , ie: 0.90, n: 0.88
Confusion:
 [[ 711   34   22]
 [  51  691   26]
 [  81  120 1454]] 



In [22]:
m= [0,1,2]
n= m/10

n

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [4]:
#Testes de algoritmos basicos na conversao numerica
clf=[]
nome=[]
clf.append(tree.DecisionTreeClassifier())
nome.append('Decision Tree')
clf.append(SVC(gamma='auto'))
nome.append('SVC Support Vector Machines')
clf.append(KNeighborsClassifier(n_neighbors=1))
nome.append('Nearest Neighbors1')
clf.append(GaussianNB())
nome.append('GaussianNB')
clf.append(SGDClassifier(loss="hinge", penalty="l2", max_iter=5, tol= 1e-3))
nome.append('SGDclassifier - Stochastic Gradient Descent')
clf.append(MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1))
nome.append('MLP classifier')
for i in range(len(clf)):
    scores = cross_val_score(clf[i], cdata, ctarg, cv=10)
    y_pred = cross_val_predict(clf[i], cdata, ctarg, cv=10)
    conf_mat = confusion_matrix(ctarg, y_pred)
    print(nome[i],'\nScores:',scores)
    print("Accuracy: %0.2f (+/- %0.2f) , ei: %0.2f , ie: %0.2f, n: %0.2f" % (scores.mean(), scores.std() * 2, conf_mat[0][0]/sum(conf_mat[0]), conf_mat[1][1]/sum(conf_mat[1]),conf_mat[2][2]/sum(conf_mat[2])))
    print('Confusion:\n', conf_mat, '\n')
    
    
clf=[]
nome=[]
clf.append(tree.DecisionTreeClassifier())
nome.append('Decision Tree')
clf.append(SVC(gamma='auto'))
nome.append('SVC Support Vector Machines')
clf.append(KNeighborsClassifier(n_neighbors=1))
nome.append('Nearest Neighbors1')
clf.append(GaussianNB())
nome.append('GaussianNB')
clf.append(SGDClassifier(loss="hinge", penalty="l2", max_iter=6, tol= 1e-3))
nome.append('SGDclassifier - Stochastic Gradient Descent')
clf.append(MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1))
nome.append('MLP classifier')
print('BAGGING')
for i in range(len(clf)):
    bclf = BaggingClassifier(clf[i], max_samples=0.5, max_features=0.5)
    scores = cross_val_score(bclf, cdata, ctarg, cv=10)
    y_pred = cross_val_predict(bclf, cdata, ctarg, cv=10)
    conf_mat = confusion_matrix(ctarg, y_pred)
    print(nome[i],'\nScores:',scores)
    print("Accuracy: %0.2f (+/- %0.2f) , ei: %0.2f , ie: %0.2f, n: %0.2f" % (scores.mean(), scores.std() * 2, conf_mat[0][0]/sum(conf_mat[0]), conf_mat[1][1]/sum(conf_mat[1]),conf_mat[2][2]/sum(conf_mat[2])))
    print('Confusion:\n', conf_mat, '\n')


print('random Forest')
clf = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(clf, cdata, ctarg, cv=10)
y_pred = cross_val_predict(clf, cdata, ctarg, cv=10)
conf_mat = confusion_matrix(ctarg, y_pred)
print('\nScores:',scores)
print("Accuracy: %0.2f (+/- %0.2f) , ei: %0.2f , ie: %0.2f, n: %0.2f" % (scores.mean(), scores.std() * 2, conf_mat[0][0]/sum(conf_mat[0]), conf_mat[1][1]/sum(conf_mat[1]),conf_mat[2][2]/sum(conf_mat[2])))
print('Confusion:\n', conf_mat, '\n')

print('AdaBoost')
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, cdata, ctarg, cv=10)
y_pred = cross_val_predict(clf, cdata, ctarg, cv=10)
conf_mat = confusion_matrix(ctarg, y_pred)
print('\nScores:',scores)
print("Accuracy: %0.2f (+/- %0.2f) , ei: %0.2f , ie: %0.2f, n: %0.2f" % (scores.mean(), scores.std() * 2, conf_mat[0][0]/sum(conf_mat[0]), conf_mat[1][1]/sum(conf_mat[1]),conf_mat[2][2]/sum(conf_mat[2])))
print('Confusion:\n', conf_mat, '\n')

print('Gradient Tree Boosting')
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
scores = cross_val_score(clf, cdata, ctarg, cv=10)
y_pred = cross_val_predict(clf, cdata, ctarg, cv=10)
conf_mat = confusion_matrix(ctarg, y_pred)
print('\nScores:',scores)
print("Accuracy: %0.2f (+/- %0.2f) , ei: %0.2f , ie: %0.2f, n: %0.2f" % (scores.mean(), scores.std() * 2, conf_mat[0][0]/sum(conf_mat[0]), conf_mat[1][1]/sum(conf_mat[1]),conf_mat[2][2]/sum(conf_mat[2])))
print('Confusion:\n', conf_mat, '\n')

print('Voting Classifier')
clf1 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf2 = AdaBoostClassifier(n_estimators=100)
clf3 = tree.DecisionTreeClassifier()
clf4 = GaussianNB()
clf5 = RandomForestClassifier(n_estimators=10)
clf = VotingClassifier(estimators=[('gbc', clf1), ('ada', clf2), ('DTc', clf3), ('gNB', clf4), ('svc', clf5)], voting='hard')
scores = cross_val_score(clf, cdata, ctarg, cv=10)
y_pred = cross_val_predict(clf, cdata, ctarg, cv=10)
conf_mat = confusion_matrix(ctarg, y_pred)
print('\nScores:',scores)
print("Accuracy: %0.2f (+/- %0.2f) , ei: %0.2f , ie: %0.2f, n: %0.2f" % (scores.mean(), scores.std() * 2, conf_mat[0][0]/sum(conf_mat[0]), conf_mat[1][1]/sum(conf_mat[1]),conf_mat[2][2]/sum(conf_mat[2])))
print('Confusion:\n', conf_mat, '\n')

Decision Tree 
Scores: [0.921875   0.93125    0.909375   0.859375   0.928125   0.85893417
 0.91222571 0.87421384 0.9022082  0.92744479]
Accuracy: 0.90 (+/- 0.05) , ei: 0.90 , ie: 0.85, n: 0.93
Confusion:
 [[ 687   38   42]
 [  39  656   73]
 [  37   76 1542]] 

SVC Support Vector Machines 
Scores: [0.859375   0.828125   0.83125    0.828125   0.88125    0.84012539
 0.89028213 0.82075472 0.85488959 0.85173502]
Accuracy: 0.85 (+/- 0.04) , ei: 0.84 , ie: 0.79, n: 0.88
Confusion:
 [[ 646   22   99]
 [  54  604  110]
 [  80  118 1457]] 

Nearest Neighbors1 
Scores: [0.621875   0.615625   0.59375    0.625      0.6875     0.63636364
 0.63322884 0.64465409 0.58990536 0.62460568]
Accuracy: 0.63 (+/- 0.05) , ei: 0.74 , ie: 0.51, n: 0.63
Confusion:
 [[ 569   55  143]
 [ 179  388  201]
 [ 386  225 1044]] 

GaussianNB 
Scores: [0.909375   0.921875   0.890625   0.875      0.934375   0.88401254
 0.92476489 0.90880503 0.86435331 0.88958991]
Accuracy: 0.90 (+/- 0.04) , ei: 0.83 , ie: 0.98, n: 0.89
Confu

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

SGDclassifier - Stochastic Gradient Descent 
Scores: [0.8125     0.646875   0.734375   0.7625     0.796875   0.77742947
 0.70532915 0.75157233 0.76025237 0.67192429]
Accuracy: 0.74 (+/- 0.10) , ei: 0.67 , ie: 0.70, n: 0.80
Confusion:
 [[ 512   94  161]
 [  65  538  165]
 [ 132  199 1324]] 

MLP classifier 
Scores: [0.659375   0.75625    0.68125    0.665625   0.51875    0.67084639
 0.6677116  0.74528302 0.52050473 0.52050473]
Accuracy: 0.64 (+/- 0.17) , ei: 0.37 , ie: 0.33, n: 0.91
Confusion:
 [[ 286   78  403]
 [  76  253  439]
 [  85   65 1505]] 

BAGGING
Decision Tree 
Scores: [0.9125     0.903125   0.9125     0.878125   0.93125    0.86833856
 0.94984326 0.89937107 0.90851735 0.93059937]
Accuracy: 0.91 (+/- 0.05) , ei: 0.91 , ie: 0.88, n: 0.94
Confusion:
 [[ 701   28   38]
 [  49  675   44]
 [  37   61 1557]] 

SVC Support Vector Machines 
Scores: [0.85       0.740625   0.740625   0.778125   0.84375    0.77429467
 0.84639498 0.80503145 0.79495268 0.85488959]
Accuracy: 0.80 (+/- 0.08)

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Usuario\Anaconda3\lib\site-

SGDclassifier - Stochastic Gradient Descent 
Scores: [0.784375   0.7125     0.653125   0.7625     0.740625   0.69278997
 0.79623824 0.78301887 0.73501577 0.74763407]
Accuracy: 0.74 (+/- 0.09) , ei: 0.70 , ie: 0.57, n: 0.85
Confusion:
 [[ 539   44  184]
 [  95  436  237]
 [ 129  122 1404]] 

MLP classifier 
Scores: [0.759375   0.725      0.771875   0.8        0.75       0.7523511
 0.73981191 0.77044025 0.80757098 0.78864353]
Accuracy: 0.77 (+/- 0.05) , ei: 0.62 , ie: 0.57, n: 0.93
Confusion:
 [[ 476   41  250]
 [  46  439  283]
 [  53   67 1535]] 

random Forest

Scores: [0.925      0.9        0.884375   0.89375    0.925      0.89028213
 0.91222571 0.88679245 0.89905363 0.91798107]
Accuracy: 0.90 (+/- 0.03) , ei: 0.89 , ie: 0.84, n: 0.93
Confusion:
 [[ 685   19   63]
 [  47  648   73]
 [  50   70 1535]] 

AdaBoost

Scores: [0.928125   0.9375     0.94375    0.921875   0.959375   0.86206897
 0.94670846 0.91509434 0.92429022 0.95583596]
Accuracy: 0.93 (+/- 0.05) , ei: 0.95 , ie: 0.90, n: 0